
# 필요한 라이브러리 설치 및 import 
1. Import and Install Dependencies

GPU를 사용하는 tensorflow와 mediapipe 설치 <br>

 mediapipe란 ? 주로 멀티 모델 오디오, 비디오 또는 모든 시계열 데이터를 구축하는 데 사용되는 프레임 워크입니다. <br>



In [2]:
# 필요한 라이브러리 설치 

#using tensorflow with GPU, mediapipe detacting movements
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# MP Holistic을 사용하여 이미지의 좌표 출력하기 

mp_holistic = mp.solutions.holistic <br>
#mp.solutions.holistic 이란 ? -> 실시간 모션인식 (얼굴,몸의 분절을 실시간으로 추적한다.)

참고 사이트 <br>
https://google.github.io/mediapipe/solutions/holistic.html



mp_drawing = 점을 연결하여 선을 보여준다.

In [4]:
# 이미지의 좌표를 추적하는 함수 만들기 
#이미지 -> RGB형식으로 만들어 모델에 대입한 결과를 results에 담고, 원상태로 만든 이미지와 results를 출력한다. 

def mediapipe_detection(image, model):   # 대상을 추적한다. 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction 모델에 대입 
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [199]:
print(results)
#<class 'mediapipe.python.solution_base.SolutionOutputs'>

results.face_landmarks.landmark
#x: 0.5144715905189514
#y: 0.24100039899349213
#z: -0.015107447281479836

#results -> <face, pose, left_hand, right_hand> 총 네 개의 결과값이 있다,
#하나의 값 (results.face_landmarks.landmark) 당 x,y,z 총 3개의 값이 3차원으로 존재한다. 

<class 'mediapipe.python.solution_base.SolutionOutputs'>


[x: 0.5144715905189514
y: 0.24100039899349213
z: -0.015107447281479836
, x: 0.5156073570251465
y: 0.21770811080932617
z: -0.02686622366309166
, x: 0.5153257846832275
y: 0.22530457377433777
z: -0.014158849604427814
, x: 0.5130105018615723
y: 0.19015543162822723
z: -0.019584190100431442
, x: 0.5160118341445923
y: 0.2093459963798523
z: -0.02846345864236355
, x: 0.5166398286819458
y: 0.19839631021022797
z: -0.02626979351043701
, x: 0.5182450413703918
y: 0.17227435111999512
z: -0.012216555885970592
, x: 0.48336973786354065
y: 0.16397900879383087
z: 0.008108302019536495
, x: 0.5193410515785217
y: 0.15384750068187714
z: -0.008623850531876087
, x: 0.519906759262085
y: 0.14311082661151886
z: -0.009443028829991817
, x: 0.5219267010688782
y: 0.10014906525611877
z: -0.0029883310198783875
, x: 0.5142927169799805
y: 0.24465233087539673
z: -0.014573230408132076
, x: 0.5141993165016174
y: 0.24743227660655975
z: -0.012925822287797928
, x: 0.5142141580581665
y: 0.2485732138156891
z: -0.01072534359991550

# 3. 위에서 얻은 results의 값을 numpy형태로 출력하기 
Extract Keypoint Values

results.pose_landmarks.landmark -> [res.x, res.y, res.z, res.visibility]으로 총 4개의 값으로 구성 <br>
그외는 <br>
results.face_landmarks.landmark -> [res.x, res.y, res.z]으로 구성 


In [7]:
# pose, face, lh, rh의 순서로 저장한다. 
# np.concatenate 를 하여 네개의 값을 하나의 넘파이 배열로 만든다.  
#만약 frame에 값이 출력되지 않았다면, (사진에서 왼손이 보이지 않아 left_hand의 result가 none으로 나올 경우) 0으로 채운다. 

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

#extract_keypoints(results).shape 
# (1662,) -> 33*4 + 468*3  + 21*3 + 21*3  

# 훈련데이터 동영상 넘파이 배열로 저장하기 
Convert Keypoint Values from videos

< 데이터 출처>  <br>
: Ai hub https://aihub.or.kr/keti_data_board/language_intelligence

<데이터 구성> <br>
데이터 크기 : 3000개의 수화 동영상 <br>
녹화 참여 인원수 : 총 10명 <br>
단어 종류 : 419개 

<데이터프레임 설명> <br>
dt : AI hub에서 가져온 동영상 파일 설명 엑셀파일 원본 <br>
dt1 : 문장과 단어가 모두 포함되어 있는 dt원본파일에서 단어 동영상만 추출 <br>
list_a : dt1의 한국어 컬럼을 단어의 개수 별로 정리한 데이터 프레임 <단어/횟수>로 구성.  

## 필요한 데이터프레임 생성 

In [ ]:
#tensorflow에서 엑셀파일 여는 라이브러리 다운 

#!pip install pandas
!pip install openpyxl

In [200]:
#dt 데이터프레임 생성 
import pandas as pd
dt=pd.read_excel("C://Users//User//sign//10.09.xlsx")
dt.head()

,번호,언어 제공자 ID,취득연도,방향,타입(단어/문장),파일명,한국어
0,1.0,1.0,2017.0,정면,단어,KETI_SL_0000000001.MOV,0.0
1,2.0,1.0,2017.0,정면,단어,KETI_SL_0000000002.MOV,1.0
2,3.0,1.0,2017.0,정면,단어,KETI_SL_0000000003.MOV,2.0
3,4.0,1.0,2017.0,정면,단어,KETI_SL_0000000004.MOV,3.0
4,5.0,1.0,2017.0,정면,단어,KETI_SL_0000000005.MOV,4.0


In [202]:
# dt 한국어컬럼 중 float 형 int형으로 바꾸기 

for i in range(len(dt['한국어'])):
    try:
        dt['한국어'][i] = int(dt['한국어'][i])
        dt['한국어'][i] = str(dt['한국어'][i])
    except:
        dt['한국어'][i] == dt['한국어'][i]

C:\Users\User\AppData\Local\Temp/ipykernel_16260/4080634729.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt['한국어'][i] = int(dt['한국어'][i])
C:\Users\User\AppData\Local\Temp/ipykernel_16260/4080634729.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt['한국어'][i] = str(dt['한국어'][i])


In [203]:
#파일명 마지막 세글자.avi로 바꾸기 
dt['파일명'] = dt['파일명'].apply(lambda x:x[:-3]) + dt['파일명'].apply(lambda x:x[-3:]).replace('MOV','avi')
dt['파일명']

0       KETI_SL_0000000001.avi
1       KETI_SL_0000000002.avi
2       KETI_SL_0000000003.avi
3       KETI_SL_0000000004.avi
4       KETI_SL_0000000005.avi
                 ...          
4699    KETI_SL_0000010371.avi
4700    KETI_SL_0000010372.avi
4701    KETI_SL_0000010373.avi
4702    KETI_SL_0000010374.avi
4703    KETI_SL_0000010375.avi
Name: 파일명, Length: 4704, dtype: object

In [204]:
## dt1 데이터 프레임 생성 
#전체 데이터 단어만 선택한 후 3000개 데이터 선택 

import pandas as pd
dt1=dt[dt['타입(단어/문장)']=='단어']
dt1 = dt1[:3000]

In [209]:
# 단어별로 동영상이 몇게 있는지 출력하는 list_a 데이터 프레임 생성 

#한국어컬럼만 추출
aa = dt1[['한국어']].copy()

#한국어 컬럼에 대한 개수 추출
list_a =aa['한국어'].value_counts()

#인덱스를 없애서 총 2개의 컬럼을 가진 데이터 프레임 생성 [한국어.횟수]
list_a =list_a.reset_index()

# 데이터프레임 컬렴명 변겅 
list_a.columns = ['한국어','횟수']
list_a = list_a.sort_values('한국어')
list_a.head()

#len(list_a)
#419

,한국어,횟수
0,0,8
22,1,8
49,10,8
328,100,7
325,1000,7


## numpy형태로 변환한 파일 담을 폴더 생성 

<폴더 구조> <br>
train 폴더 <br>
단어의 뜻을 이름으로 하는 폴더 419개  <br>
각각의 단어 폴더 안에 단어 당 촬영한 동영상 개수 만큼 폴더 생성   <br>
각각의 동영상 개수 폴더 안에 프레임 횟수만큼 폴더를 만들고 하나의 폴더안에 프레임 하나의 넘파이 배열이 담긴 파이썬 파일 첨부 

In [213]:
# 단어별 -동영상개수만큼의 폴더 생성하기 
import os
# train 폴더 생성
os.mkdir( 'C://Users//User//sign//train')

# 단어별 폴더 생성 
for k in range(419): 
    a = 'C://Users//User//sign//train//' + str(list_a['한국어'][k])  # 동영상 이름 
    os.mkdir(a)
    # 단어 폴더 안에 동영상의 횟수만큼 폴더 생성 
    for i in range(list_a['횟수'][k]):
        b= 'C://Users//User//sign//train//' + str(list_a['한국어'][k]) + '//' + str(i)
        os.mkdir(b)

## video3000에 있는 동영상 읽어와서 numpy배열로 변환한 후 위에서 만든 train 폴더에 저장 


In [ ]:
#폴더의 개수 출력하는 함수 생성 
def dir_empty(dir_path):
    return len(os.listdir(dir_path))



# holistic 모델 사용 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # 3000개의 동영상 분석 
    for k in range(1000):
        # dt1['파일명']의 동영상 3000개 분석 
        #dt1 -> 단어별로 정리한 동영상 파일 3000개 
        
        videos = "C://Users//User//sign//video3000//" + dt1['파일명'][k] 

        
        #openCV를 열고 동영상을 읽어들인다. -> 동영상의 처음부터 끝까지 frame형태로 기록된다. 
        # -> 출력물 ret. frame 
        
        # ret이란 frame이 존재하는지 존재하지 않는지 True, False로 출력한다. 
        # 동영상이 끝나 더이상의 frame이 존재하지 않으면 False출력 
        
        cam = cv2.VideoCapture(videos)
        
        #출력하는 동영상의 해당 단어 kor에 담기 
        kor = dt1['한국어'][k]
        
        # videocount -> 해당 단어의 몇번째 동영상인지 출력 
        videocount =0

      # 
        while (True):
            #만약 해당 단어 폴더의 vidoecount의 이름을 가진 폴더의 길이가 0이라면(비어있다면)
            #  해당단어 - video count - frame 개수 별로 만든 폴더에 넘파이 파일 저장 
            if dir_empty('C://Users//User//sign//train//' + kor + '//'+str(videocount)) == 0:
                
                #프레임 개수를 알려주는 currentframe 
                currentframe = 0

                while (True):
                    # 여기서의 True 가 밑의 ret == True로 연결
                    #ret 이 True 라면 계속 반복, 즉 프레임이 남아있다면 계속해서 읽어들인다. 



                    # 위에서 읽은 cam을 frame별로 하나씩 읽어들이겠다.
                    ret,frame = cam.read()


                    # 만약 프레임이 남아있다면 
                    if ret == True:
                        
                        image, results = mediapipe_detection(frame, holistic)  # frame에 있는 점들을 추적하여 result에 넣는다. 
                        keypoints = extract_keypoints(results) #result를 하나의 넘파이 배열로 만든다
                        
                        # 프레임 번호를 폴더명으로 하는 폴더 생성 
                        os.mkdir('C://Users//User//sign//train//' + kor + '//'+str(videocount)+'//'+ str(currentframe))
                        
                        #위에서 만든 폴더에 넘파이 배열 파일을 넣겠다. 
                        npy_path = 'C://Users//User//sign//train//' + kor + '//'+str(videocount)+'//'+ str(currentframe)+'//' +str(currentframe)
                        np.save(npy_path, keypoints)
                        
                        #다음프레임으로 이동 
                        currentframe +=1
                        


                    #만약 프레임이 끝났다면 즉, 하나의 동영상이 완료되었다면 종료 (두번째 while 문 종료)
                    else:
                        break
                
                # 하나의 동영상 추출 완료 ( if문 종료 )
                # dir_empty('C://Users//User//sign//train//' + kor + '//'+str(videocount)) == 0: 종료 
                # 완료를 알려주는 문장 출력 
                print(kor + '단어' + str(currentframe) + '프레임 추출 완료======================================')
                
                break
                
    #첫번째 while 문 
    #만약 ir_empty('C://Users//User//sign//train//' + kor + '//'+str(videocount)) == 0 이 아니라면, 
    # 즉 단어 - 동영상 횟수를 나타내는 폴더가 비어있지 않다면 다음폴더로 이동 
    #ex ) 사자- 1번째 동영상의 폴더의 길이가 0이아니라면 사자-2 번째 폴더로 이동하고 위의 문장을 다시 실행 
    
            else:
                videocount +=1
                continue

0단어141프레임 추출 완료======================================
1단어160프레임 추출 완료======================================
2단어123프레임 추출 완료======================================
3단어146프레임 추출 완료======================================
4단어144프레임 추출 완료======================================
5단어151프레임 추출 완료======================================
6단어165프레임 추출 완료======================================
7단어148프레임 추출 완료======================================
8단어137프레임 추출 완료======================================
9단어147프레임 추출 완료======================================
10단어148프레임 추출 완료======================================
11단어147프레임 추출 완료======================================
12단어131프레임 추출 완료======================================
13단어137프레임 추출 완료======================================
14단어156프레임 추출 완료======================================
15단어164프레임 추출 완료======================================
16단어173프레임 추출 완료======================================
17단어141프레임 추출 완료======================================
18단어156프레임 추출 완료====

In [ ]:
## 3차원 형태의 넘파이 배열을 모델에 넣기 위해서는 모든 프레임의 개수가 같아야 한다.
# 프레임의 개수가 가장 많은 영상을 찾아 그 영상에 맞춰 모든 영상의 마지막 프레임을 해당 프레임의 개수만큼 복사한다. 

# 6. 데이터 라벨링하기 


### 동영상 데이터 라벨링하기


In [7]:
!pip uninstall protobuf
#pip install --no-binary=protobuf protobuf

^C


In [8]:
# 다른 액션에 대한 레이블 만들기 
# 필요한 데이터 임포트
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

TypeError: Couldn't build proto file into descriptor pool!
Invalid proto descriptor for file "tensorflow/python/framework/cpp_shape_inference.proto":
  tensorflow.CppShapeInferenceResult.HandleShapeAndType.specialized_type: ".tensorflow.SpecializedType" is not defined.


In [ ]:
# 정답데이터 라벨링 하기 
label_map = {label:num for num, label in enumerate(dt1['한국어'])} # enumerate-> 0 부터 순서대로 나타내 주는 것 

In [ ]:
label_map

In [ ]:
##### 프레임 개수만 맞추면 됩니다. 

In [ ]:
# 파일 불러와서 해당 영상에 대한 정답 데이터 labels에 담기 
# labeling actions to labels 

sequences, labels = [], []
DATA_PATH  =  'C://Users//User//sign//train//'
for action in dt1['한국어']:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int): #단어 안에 있는 폴더의 이름을 리스트로 출력하고 int형으로 변경 
        window = []
        for currentframe in range(****sequence_length(프레임 개수****)):
            # 데이터 파이썬 파일로 불러오기 
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(currentfrmae)))
            # 리스트에 append 하기 
            window.append(res)
            #sequence라는 리스트안에 window가 리스트 형터로 존재 
            #리스트 안의 리스트 
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.array(sequences).shape
# 총 90개의 비디오, 1편 당 30시퀀스 /  # (1662,)키포인트 -> 468*3 + 33*4 + 21*3 + 21*3 각(xyz 즉 세개씩 곱한다.)


In [ ]:
np.array(labels).shape  #(90,) 라벨링한 정답 데이터 

In [ ]:
X = np.array(sequences)

In [ ]:
X.shape
#90 channels, 30 frames, 1662 keypoints (68*3 + 33*4 + 21*3 + 21*3)

In [ ]:
# 원핫인코딩 하기 
# one hot encoding 
y = to_categorical(labels).astype(int)

In [ ]:
# 데이터 나누기
# split data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05) # 5%가 테스트 되겠다

In [ ]:
y_test.shape

# LSTM Neural Network 만들기 

Build and Train LSTM Neural Network

In [ ]:
# 모델 생성 -> cnn 넣었을 때, 넣지 않았을 때 모두 해보기 

In [ ]:
# 모델 임포트 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense  #LSTM layer 를  쓰겠다.  
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
# set call back ( prevent over fitting )
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
# 이 타입 쓰는 이유 CNN 과 비교 시 장점 
# 미디아 type + LSTM 레이어
# 1. Less DaTa Required 
# 2. Faster To train
# 3. Faster Detections 

In [ ]:
model = Sequential()  # 시작 
# 30, 1662) 각각의 비디오 30 시퀀스 1662 키포인트


#######
# cnn 모델 추가 

########

model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662))) #64 유닛 -> LSTM은 다음 레이어에 sequences가 필요해서 다음 레이어가 있으면 True를 해준다
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))# 다음은 dense슬라이드라 sequences 필요없어
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax')) # ctions.shape[0] -> 3네트워크 유닛 

In [ ]:
# 아담 그레이드 , 비아너리 -> 바이너리 크로스 / 이건 분류하는 거-> multi class categoty  예측하는 확률을 보여준다. 
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

In [ ]:
model.summary()

 -  결과보기 
 - visualize the result of model



 cmd 창을 킨다 <br>
open cmd  <br>
<br>
C:\Users\User>cd Logs <br>
<br>
C:\Users\User\Logs>cd Train<br>
<br>
C:\Users\User\Logs\train>dir<br>

- 파일을 텐서플로우로 실행시킨다. (run the file )
C:\Users\User\Logs\train>tensorboard --logdir=. 

- 링크를 복사하여 크롬창에 띄운다 ( copy the like and open it at Chrome)

# 예측하기 
8. Make Predictions

In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(res[4])]

In [ ]:
actions[np.argmax(y_test[4])]

# 모델 저장 
9. Save Weights

In [ ]:
model.save('action.h5')

In [ ]:
model.load_weights('action.h5')

# 모델 평가 
10. Evaluation using Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_train)

In [ ]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)
# 이원교차표 출력 

In [ ]:
accuracy_score(ytrue, yhat)

# 실시간 예측하기 
11. Test in Real Time

In [5]:
# 동영상에 점 찍기 

def draw_landmarks(image, results):  # 동영상의 frame과 동영상에서 받은 result를 받는다. 
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # 얼굴의 점을 연결하여 그린다.
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # POSE를 연결햐여 그린다.
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # 왼손 그리기
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # 오른손 그리기 

In [6]:
# 더 정확한 판단을 위해 각각의 노드에 두개의 점을 추가한다. 
#add aditional 2 spots to the image

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),  
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
from scipy import stats

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
# 1. New detection variables #30개 차면 detect 할 수 있게
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:] #마지막 30프레임
        
        if len(sequence) == 30: #30개차면 predict한다 
            res = model.predict(np.expand_dims(sequence, axis=0))[0] # 2차원을 3차원으로 늘려준다. [0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()